In [8]:
"""
Landcover assessments for MODIS and VIIRS AFDs
Author: maxwell.cook@colorado.edu
"""

import os, gc
import geopandas as gpd
import rioxarray as rxr

import warnings
warnings.filterwarnings("ignore") # suppresses annoying geopandas warning

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [4]:
# Load the MODIS and VIIRS AFDs (buffered)

modis_afd = os.path.join(projdir,f'data/spatial/mod/AFD/MOD61-afd_fired-aspen_2018_to_2023_buffer.gpkg')
snpp_afd = os.path.join(projdir,f'data/spatial/mod/AFD/SNPP-afd_fired-aspen_2018_to_2023_buffer.gpkg')

afds = {
    'MOD61': gpd.read_file(modis_afd),
    'SNPP': gpd.read_file(snpp_afd)
}

afds['MOD61'].head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,...,FRP,DAYNIGHT,TYPE,id,ig_date,last_date,ACQ_MONTH,ACQ_YEAR,ACQ_DATETIME,geometry
0,37.5025,-112.7346,300.2,1.2,1.1,2018-01-12,0548,Terra,MODIS,22,...,12.0,N,0,44975,2018-01-26,2018-02-02,1,2018,2018-01-12 05:48:00+00:00,"POLYGON ((-1457354.388 1736257.445, -1457354.3..."
1,38.5271,-111.9255,320.4,2.2,1.4,2018-01-25,1856,Terra,MODIS,76,...,74.9,D,0,43657,2018-01-25,2018-01-30,1,2018,2018-01-25 18:56:00+00:00,"POLYGON ((-1368392.364 1837372.349, -1368392.3..."
2,38.5212,-111.8990,319.6,2.2,1.4,2018-01-25,1856,Terra,MODIS,0,...,72.3,D,0,43657,2018-01-25,2018-01-30,1,2018,2018-01-25 18:56:00+00:00,"POLYGON ((-1366245.947 1836338.869, -1366245.9..."
3,38.5227,-111.9221,343.4,1.0,1.0,2018-01-25,2036,Aqua,MODIS,93,...,53.8,D,0,43657,2018-01-25,2018-01-30,1,2018,2018-01-25 20:36:00+00:00,"POLYGON ((-1368185.097 1836837.103, -1368185.0..."
4,38.5155,-111.9088,343.1,1.0,1.0,2018-01-25,2036,Aqua,MODIS,93,...,53.1,D,0,43657,2018-01-25,2018-01-30,1,2018,2018-01-25 20:36:00+00:00,"POLYGON ((-1367186.979 1835849.890, -1367186.9..."


In [ ]:
# Load the landfire EVT data

In [9]:
evt_fp = os.path.join(maindir,'data/landcover/LANDFIRE/LF2016_EVT_200_CONUS/Tif/LC16_EVT_200.tif')
evt_da = rxr.open_rasterio(evt_fp, masked=True, cache=False, chunks='auto').squeeze()
shp, gt, wkt, nd = evt_da.shape, evt_da.spatial_ref.GeoTransform, evt_da.rio.crs, evt_da.rio.nodata
print(
    f"Shape: {shp}; \n"
    f"GeoTransform: {gt}; \n"
    f"WKT: {wkt}; \n"
    f"NoData Value: {nd}; \n"
    f"Data Type: {evt_da[0].dtype}")
gc.collect()

Shape: (97283, 154207); 
GeoTransform: -2362425.0 30.0 0.0 3177435.0 0.0 -30.0; 
WKT: EPSG:5070; 
NoData Value: nan; 
Data Type: float32


1330

In [ ]:
# Run zonal statistics for the AFD buffers